In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy.optimize import brentq, fsolve, least_squares
from scipy.interpolate import interp2d
from scipy.integrate import quad
from scipy.stats import norm

import warnings
warnings.filterwarnings('ignore')

In [2]:
ois_disc_factor = pd.read_excel('Data_1_Output.xlsx', sheet_name='oisDiscFactors')['DiscountFactor']
fwd_swap_rates = pd.read_excel('Data_1_Output.xlsx', sheet_name='fwdSwapRates')

In [3]:
### Reading and cleaning swaption mkt data
swaption_data = pd.read_excel("IR Data.xlsx", sheet_name='Swaption', header =2)

# Change to pct
swaption_data.iloc[:, 2:] = swaption_data.iloc[:, 2:]/100

swaption_data

,Expiry,Tenor,-200bps,-150bps,-100bps,-50bps,-25bps,ATM,+25bps,+50bps,+100bps,+150bps,+200bps
0,1Y,1Y,0.91570,0.62030,0.44130,0.31224,0.26182,0.2250,0.2096,0.2140,0.2434,0.27488,0.30297
1,1Y,2Y,0.83270,0.61240,0.46570,0.35807,0.31712,0.2872,0.2712,0.2684,0.2851,0.31025,0.33523
2,1Y,3Y,0.73920,0.56870,0.44770,0.35745,0.32317,0.2978,0.2829,0.2780,0.2877,0.30725,0.32833
3,1Y,5Y,0.55190,0.44640,0.36510,0.30242,0.27851,0.2607,0.2498,0.2456,0.2512,0.26536,0.28165
4,1Y,10Y,0.41180,0.35040,0.30207,0.26619,0.25351,0.2447,0.2398,0.2382,0.2425,0.25204,0.26355
5,5Y,1Y,0.67800,0.49090,0.38400,0.31485,0.29060,0.2726,0.2604,0.2532,0.2494,0.25320,0.25980
6,5Y,2Y,0.57880,0.46410,0.39033,0.33653,0.31531,0.2983,0.2856,0.2765,0.2671,0.26540,0.26760
7,5Y,3Y,0.53430,0.44440,0.38180,0.33437,0.31536,0.2998,0.2876,0.2782,0.2667,0.26200,0.26150
8,5Y,5Y,0.41990,0.36524,0.32326,0.29005,0.27677,0.2660,0.2573,0.2502,0.2406,0.23570,0.23400
9,5Y,10Y,0.34417,0.30948,0.28148,0.25954,0.25136,0.2451,0.2399,0.2356,0.2291,0.22490,0.22250


In [4]:
libor_forward_rate = fwd_swap_rates["liborForwardSwapRate"]

In [5]:
date_count_df = fwd_swap_rates[['Start','Tenor']]
date_count_df = date_count_df.apply(lambda x: x.str.rstrip('Y')).copy()
date_count_df

,Start,Tenor
0,1,1
1,1,2
2,1,3
3,1,5
4,1,10
5,5,1
6,5,2
7,5,3
8,5,5
9,5,10


In [6]:
pvbp = []
for i in range(len(date_count_df)):
    expiry = date_count_df['Start'][i]
    T = date_count_df['Tenor'][i]
    
    pvbp.append(0.5 * sum(ois_disc_factor[2*int(expiry): (int(expiry)+int(T))*2]))
pd.DataFrame(pvbp)

,0
0,0.996139
1,1.988791
2,2.977832
3,4.943924
4,9.771149
5,0.980959
6,1.957022
7,2.928051
8,4.853642
9,9.569368


In [7]:
bps = np.array([-0.02, -0.015, -0.01, -0.005, -0.0025, 0, 0.0025, 0.005, 0.01, 0.015, 0.02])

strike = []
for i in range(len(libor_forward_rate)):
    strike.append(libor_forward_rate[i] + bps)

### Black Call Function

In [ ]:
def calculateBlackCall(S, K, T, sigma, pvbp):
    d1 = (np.log(S/K)+ 0.5*(sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    blackCallPrice = pvbp * (S*norm.cdf(d1) - K*norm.cdf(d2))
    return blackCallPrice

### Black Put Function

In [9]:
def calculateBlackPut(S, K, T, sigma, pvbp):
    d1 = (np.log(S/K)+0.5*(sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    blackPutPrice = pvbp * (K*norm.cdf(-d2) - S*norm.cdf(-d1))
    return blackPutPrice

### DD Call Function

In [10]:
def calculateDDCall(S, K, T, pvbp, sigma, beta):
    S_d = S/beta
    K_d = K + ((1-beta)/beta)*S
    sigma_d = sigma*beta
    ddCallPrice = calculateBlackCall(S_d, K_d, T, sigma_d, pvbp)
    return ddCallPrice

### DD Put Function

In [11]:
def calculateDDPut(S, K, T, pvbp, sigma, beta):
    S_d = S/beta
    K_d = K + ((1-beta)/beta)*S
    sigma_d = sigma*beta
    ddPutPrice = calculateBlackPut(S_d, K_d, T, sigma_d, pvbp)
    return ddPutPrice

### Calculate DD volatility

In [12]:
def DDvolatility(F, K, pvbp, T, sigma, beta):
    if F <= K:
        price = calculateDDCall(F, K, T, pvbp, sigma, beta)
        vol = fsolve(lambda x: price - calculateBlackCall(F, K, T, x, pvbp), 0.5)

    else:
        price = calculateDDPut(F, K, T, pvbp, sigma, beta)
        vol = fsolve(lambda x: price - calculateBlackPut(F, K, T, x, pvbp), 0.5)

    return vol[0]

### Calculate DD calibration

In [13]:
def DDcalibration(x, K, vols, pvbp, F, T):
    err = 0.0
    sigma = vols[5]

    for i, vol in enumerate(vols):
        err += (vol - DDvolatility(F, K[i], pvbp, T, sigma, x))**2

    return err

### Compute Sigma and Beta values

In [14]:
start = date_count_df.Start
tenor = date_count_df.Tenor

beta_results = []
initial_guess = [0.3]

for i in range(len(pvbp)):
    results = least_squares(lambda x: DDcalibration(x, strike[i], swaption_data.values[i][2:],
                                                    pvbp[i], libor_forward_rate[i], T = int(start[i])),
                            initial_guess, bounds = ([0,1]))

    beta_results.append(results.x[0])

In [15]:
sigma_beta_df = pd.concat([pd.DataFrame(swaption_data.iloc[:,7].values), pd.DataFrame(beta_results)], axis=1, ignore_index=True)
sigma_beta_df = sigma_beta_df.rename(columns={0:'Sigma',1:'Beta'})
sigma_beta_df

,Sigma,Beta
0,0.2250,3.195608e-08
1,0.2872,3.142889e-13
2,0.2978,3.891706e-07
3,0.2607,2.418872e-06
4,0.2447,6.655326e-06
5,0.2726,9.851157e-07
6,0.2983,6.314974e-08
7,0.2998,2.277158e-06
8,0.2660,1.432547e-04
9,0.2451,5.546197e-02


# Compute SABR Model

In [16]:
def SABR(F, K, T, alpha, rho, nu, beta):
    X = K
    # if K is at-the-money-forward
    if abs(F - K) < 1e-12:
        numer1 = (((1 - beta)**2)/24)*alpha*alpha/(F**(2 - 2*beta))
        numer2 = 0.25*rho*beta*nu*alpha/(F**(1 - beta))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        VolAtm = alpha*(1 + (numer1 + numer2 + numer3)*T)/(F**(1-beta))
        sabr_sigma = VolAtm
    else:
        z = (nu/alpha)*((F*X)**(0.5*(1-beta)))*np.log(F/X)
        zhi = np.log((((1 - 2*rho*z + z*z)**0.5) + z - rho)/(1 - rho))
        numer1 = (((1 - beta)**2)/24)*((alpha*alpha)/((F*X)**(1 - beta)))
        numer2 = 0.25*rho*beta*nu*alpha/((F*X)**((1 - beta)/2))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        numer = alpha*(1 + (numer1 + numer2 + numer3)*T)*z
        denom1 = ((1 - beta)**2/24)*(np.log(F/X))**2
        denom2 = (((1 - beta)**4)/1920)*((np.log(F/X))**4)
        denom = ((F*X)**((1 - beta)/2))*(1 + denom1 + denom2)*zhi
        sabr_sigma = numer/denom

    return sabr_sigma

In [17]:
def sabr_calibration(F, strikes, T, market_vol, x, beta):
    err = 0.0
    for vol,K in zip(market_vol, strikes):
        err += (vol - SABR(F, K, T, x[0], x[1], x[2], beta))**2
    return err

In [18]:
start = date_count_df.Start
tenor = date_count_df.Tenor

alpha_results = []
rho_results = []
nu_results = []

initial_guess = [0.1,-0.1,0.1]

for i in range(len(pvbp)):
    results = least_squares(lambda x: sabr_calibration(libor_forward_rate[i], strike[i], int(start[i]), swaption_data.values[i][2:], x, 0.9), initial_guess, bounds = ([0.0,-1.0,0.0], [np.inf, 1, np.inf]))

    alpha_results.append(results.x[0])
    rho_results.append(results.x[1])
    nu_results.append(results.x[2])


sabr_calibration_df = pd.concat([pd.DataFrame(alpha_results, columns=['Alpha']),pd.DataFrame(rho_results, columns=['Rho']),pd.DataFrame(nu_results, columns=['Nu'])], axis=1)

sabr_calibration_df

,Alpha,Rho,Nu
0,0.139065,-0.633248,2.049566
1,0.184646,-0.525118,1.677438
2,0.196851,-0.482847,1.438131
3,0.178046,-0.414397,1.064941
4,0.171009,-0.264055,0.778717
5,0.166505,-0.585049,1.339434
6,0.199232,-0.545653,1.061047
7,0.209696,-0.547185,0.935995
8,0.187755,-0.490747,0.681123
9,0.172283,-0.387881,0.525583


### Find Libor Forward Swap (2x10) and (8x10)

In [19]:
forward_swaps_2x10_8x10 = \
    [(2,10, 0.039634),
     (8,10, 0.048711)]

forward_swaps_df_2x10_8x10 = pd.DataFrame(columns=["Start","Tenor","liborForwardSwapRate"])
rows_list_part_2 = []

for swap_combination in forward_swaps_2x10_8x10:
    swap_start_special = swap_combination[0]
    swap_tenor_special = swap_combination[1]
    swap_rate_special = swap_combination[2]
    new_row = pd.DataFrame({'Start':[str(swap_start_special)+"Y"], 'Tenor':[str(swap_tenor_special)+"Y"], 'liborForwardSwapRate':[swap_rate_special]})
    rows_list_part_2.append(new_row)

forward_swaps_df_2x10_8x10 = pd.concat(rows_list_part_2, ignore_index=True)
forward_swaps_df_2x10_8x10

,Start,Tenor,liborForwardSwapRate
0,2Y,10Y,0.039634
1,8Y,10Y,0.048711


### Pricing swaption with DD and SABR model

In [20]:
payer_swaption_strikes = [0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08]
receiver_swaption_strikes = [0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08]
payer_expiry = 2
payer_tenor = 10
receiver_expiry = 8
receiver_tenor = 10

#Calculated from discount factors
payer_forward = forward_swaps_df_2x10_8x10['liborForwardSwapRate'][0]
receiver_forward = forward_swaps_df_2x10_8x10['liborForwardSwapRate'][1]

### Data interpolation

In [21]:
#Interpolate the calibrated parameters
expiries = np.array([1, 5, 10])
tenors = np.array([1, 2, 3, 5, 10])
DD_sigma = [list(sigma_beta_df.iloc[0:5,0].values),list(sigma_beta_df.iloc[5:10,0].values),list(sigma_beta_df.iloc[10:15,0].values)]
DD_Beta = [list(sigma_beta_df.iloc[0:5,1].values),list(sigma_beta_df.iloc[5:10,1].values),list(sigma_beta_df.iloc[10:15,1].values)]
SABR_alpha = [list(sabr_calibration_df.iloc[0:5,0].values),list(sabr_calibration_df.iloc[5:10,0].values),list(sabr_calibration_df.iloc[10:15,0].values)]
SABR_rho = [list(sabr_calibration_df.iloc[0:5,1].values),list(sabr_calibration_df.iloc[5:10,1].values),list(sabr_calibration_df.iloc[10:15,1].values)]
SABR_nu = [list(sabr_calibration_df.iloc[0:5,2].values),list(sabr_calibration_df.iloc[5:10,2].values),list(sabr_calibration_df.iloc[10:15,2].values)]


#beta and sigma
def interpolate_dd_parameters(t, e, tenors, expiries):
    f_linear_beta = interp2d(tenors, expiries, DD_Beta, kind='linear')
    f_linear_sigma = interp2d(tenors, expiries, DD_sigma, kind='linear')
    interpolated_beta_linear = f_linear_beta(t, e)
    interpolated_sigma_linear = f_linear_sigma(t, e)

    return interpolated_beta_linear[0], interpolated_sigma_linear[0]

#alpha, rho and nu
def interpolate_sabr_parameters(t, e, tenors, expiries):
    f_linear_alpha = interp2d(tenors, expiries, SABR_alpha, kind='linear')
    f_linear_rho = interp2d(tenors, expiries, SABR_rho, kind='linear')
    f_linear_nu = interp2d(tenors, expiries, SABR_nu, kind='linear')
    interpolated_alpha_linear = f_linear_alpha(t, e)
    interpolated_rho_linear = f_linear_rho(t, e)
    interpolated_nu_linear = f_linear_nu(t, e)

    return interpolated_alpha_linear[0], interpolated_rho_linear[0], interpolated_nu_linear[0]

DD_Beta

[[3.1956079337390475e-08,
  3.142889462060772e-13,
  3.8917060966600223e-07,
  2.418872081806822e-06,
  6.6553260105834885e-06],
 [9.851156869647574e-07,
  6.31497354742053e-08,
  2.2771584741820925e-06,
  0.0001432546507178509,
  0.05546197005976203],
 [1.3949423753015396e-07,
  7.488150489023309e-06,
  8.153753205824166e-05,
  1.3874103128126355e-06,
  0.0017444231220399044]]

### Payer Swaption

In [22]:
beta, sigma = interpolate_dd_parameters(10, 2, tenors, expiries)
alpha, rho, nu = interpolate_sabr_parameters(10, 2, tenors, expiries)

In [24]:
payer = pd.DataFrame(index=payer_swaption_strikes,columns=['DD Price', 'SABR Price'])
present_value_2_10 = 0.5 * sum(ois_disc_factor[2*int(2): (int(2)+int(10))*2])

for K in payer_swaption_strikes:
    payer.loc[K]['DD Price'] = calculateDDCall(payer_forward, K, 2, present_value_2_10, sigma, beta)

for K in payer_swaption_strikes:
    SABR_vol = SABR(payer_forward, K, 2, alpha, rho, nu, 0.9)
    payer.loc[K]['SABR Price'] = calculateBlackCall(payer_forward, K, 2, SABR_vol, present_value_2_10)

In [25]:
payer.index.names = ['Strike (%)']
payer

,DD Price,SABR Price
Strike (%),,
0.01,0.28886,0.29035
0.02,0.195421,0.198782
0.03,0.112604,0.115315
0.04,0.051471,0.05208
0.05,0.017408,0.021508
0.06,0.004116,0.010998
0.07,0.000653,0.00689
0.08,0.000068,0.004892


### Receiver Swaption

In [26]:
beta_10, sigma_10 = interpolate_dd_parameters(10, 8, tenors, expiries)
alpha_10, rho_10, nu_10 = interpolate_sabr_parameters(10, 8, tenors, expiries)

beta_10

0.023231441897128755

In [27]:
receiver = pd.DataFrame(index=payer_swaption_strikes,columns=['DD Price', 'SABR Price'])
present_value_8_10 = 0.5 * sum(ois_disc_factor[2*int(8): (int(8)+int(10))*2])

for K in receiver_swaption_strikes:
    receiver.loc[K]['DD Price'] = calculateDDPut(payer_forward, K, 8, present_value_8_10, sigma_10, beta_10)

for K in receiver_swaption_strikes:
    SABR_vol = SABR(payer_forward, K, 2, alpha, rho, nu, 0.9)
    receiver.loc[K]['SABR Price'] = calculateBlackPut(payer_forward, K, 8, SABR_vol, present_value_8_10)

In [28]:
receiver.index.names = ['Strike (%)']
receiver

,DD Price,SABR Price
Strike (%),,
0.01,0.017801,0.025094
0.02,0.035281,0.044731
0.03,0.063444,0.067738
0.04,0.10446,0.104293
0.05,0.159021,0.167221
0.06,0.22611,0.249104
0.07,0.303393,0.338007
0.08,0.387989,0.429509


### Export Datasets

In [29]:
# with pd.ExcelWriter("Data_2_Output.xlsx") as writer:
#     pd.concat([date_count_df,pd.DataFrame(pvbp, columns=['PVBP'])], axis=1).to_excel(writer, sheet_name='PVBP',index=True)
#     pd.concat([date_count_df,sabr_calibration_df], axis=1).to_excel(writer, sheet_name='SABR_Results',index=True)
#     pd.concat([date_count_df,sigma_beta_df], axis=1).to_excel(writer, sheet_name='DD_Results',index=True)

In [30]:
# with pd.ExcelWriter("Data_3_Output.xlsx") as writer:
#     payer.to_excel(writer, sheet_name='Payer_Swaption',index=True)
#     receiver.to_excel(writer, sheet_name='Receiver_Swaption',index=True)